# Visualizing the performance of deep learning models with `Tensorboard`

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).

`TensorBoard` is a web-based visualization tool that comes bundled with `TensorFlow`. It allows developers and researchers to visualize and monitor their `TensorFlow` models and experiments.

With `TensorBoard`, you can visualize a wide range of data, such as scalar values, images, and histograms. You can monitor the training progress of your model, including the loss and accuracy over time, and also visualize the gradients and weights of the model. This can help you identify potential issues with your model and optimize its performance.

`TensorBoard` also supports interactive visualization, which allows you to explore your model and data in real-time. For example, you can visualize embeddings in high-dimensional space and interactively explore clusters of similar data points. In addition to its visualization capabilities, `TensorBoard` also allows you to compare and track multiple experiments, making it easy to compare the performance of different models and configurations.

<img src="https://www.tensorflow.org/tensorboard/images/tensorboard.gif" width=400 />

In this notebook, we will show you how to set up your `TensorBoard`, which for many developers is the primary control board of ML experiments.

If you don't have tensorboard installed, you can quickly get this package by:

- `pip install tensorboard` in your terminal/CLI.
- `%pip install tensorboard` in a code cell of your jupyter notebook.
- `!pip install tensorboard` in a code cell of your colab notebook.

Now, you just need to load the tensorboard into your Notebook and set a log directory (_this directory is where tensorboard will store all the logs from your ML experiments_).



In [1]:
%load_ext tensorboard
log_folder = 'logs'

If you want to reload the tensorboard from before:

- `%reload_ext tensorboard`.

If you want to clear the current logs to write new ones:

- `rm -rf logs`.

If you want to add a timestamp to your logs:

- `log_folder = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")`.


In this example, we will use the `IMDB sentiment analysis dataset.

<img src="https://upload.wikimedia.org/wikipedia/commons/6/69/IMDB_Logo_2016.svg" width=400 />

`IMDB sentiment analysis` is a popular natural language processing task that involves analyzing movie reviews to determine whether they are positive or negative (a binary classification problem). The dataset used for this task consists of thousands of movie reviews from the Internet Movie Database (`IMDB`), labeled as either positive or negative. 

The goal of sentiment analysis is to build a model that can accurately classify these reviews based on their sentiment. This task is important because it has many practical applications, such as helping companies monitor customer feedback and reviews.

This will be one of our first implementations of a `NLP` model. `NLP`, or Natural Language Processing, is a subfield of artificial intelligence and computational linguistics that focuses on enabling computers to understand, interpret, and generate human language. `NLP` involves building models and algorithms that can analyze and process human language in various forms, such as text, speech, and images. Some of the most common applications of `NLP` include `sentiment analysis`, `machine translation`, `speech recognition`, `text classification`, `text generation`, and chatbots.


In [2]:
import tensorflow as tf
from keras import layers
from keras.datasets import imdb
from keras_preprocessing.sequence import pad_sequences


vocabulary = 2000 # Number of words to consider as features.
max_len = 500 # Cuts off texts after this number of words.

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocabulary)

x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

model = tf.keras.models.Sequential()

model.add(layers.Embedding(vocabulary, 128,
                            input_length=max_len,
                            name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))

model.compile(optimizer='rmsprop',
            loss='binary_crossentropy',
            metrics=['acc'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed (Embedding)           (None, 500, 128)          256000    
                                                                 
 conv1d (Conv1D)             (None, 494, 32)           28704     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 98, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 92, 32)            7200      
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 3

Now that you have a model to track, you need to set your `Tensorboard` callback during the model’s fit method (this is the method responsible for logging events such as activation histograms, metrics plots, training graph visualizations).

Arguments that the `TensorBoard callback` takes are:

- `log_dir`: where to store the logs.
- `histogram_freq`: the frequency at which to compute activation and weight histograms for layers of the model (_0 means that histograms will not be computed_).
- `write_graph`: `True` will make the graph vizualization in TensorBoard.
- `write_images`: `True` will make model weights be visualized as an image in TensorBoard.
- `update_freq`: determines how losses and metrics are written to TensorBoard (_when set to `epoch` they are written after every epoch_).
- `profile_batch`: determines which batches will be profiled (_by default, the second batch is profiled. Setting profile_batch to 0 disables profiling_).
- `embeddings_freq`: the frequency at which the embedding layers will be visualized (_setting to 0 means that the embeddings will not be visualized_).


In [3]:

callbacks = [tf.keras.callbacks.TensorBoard(log_dir=log_folder,
                                           histogram_freq=1,
                                           write_graph=True,
                                           write_images=True,
                                           update_freq='epoch',
                                           profile_batch=2,
                                           embeddings_freq=1)]

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128,
                    validation_split= 0.2,
                    callbacks=callbacks,
                    verbose=1)

test_loss_score, test_acc_score = model.evaluate(x_test, y_test)

print(f'Final Loss: {round(test_loss_score, 2)}.')
print(f'Final Performance: {round(test_acc_score * 100, 2)} %.')

Version:  2.10.1
Eager mode:  True
GPU is available
Epoch 1/20
157/157 [==============================] - 9s 32ms/step - loss: 0.5866 - acc: 0.6872 - val_loss: 0.4315 - val_acc: 0.8344
Epoch 2/20
157/157 [==============================] - 4s 28ms/step - loss: 0.4242 - acc: 0.8540 - val_loss: 0.4342 - val_acc: 0.8602
Epoch 3/20
157/157 [==============================] - 4s 28ms/step - loss: 0.3546 - acc: 0.8838 - val_loss: 0.5057 - val_acc: 0.8522
Epoch 4/20
157/157 [==============================] - 4s 28ms/step - loss: 0.3240 - acc: 0.9036 - val_loss: 0.5703 - val_acc: 0.8570
Epoch 5/20
157/157 [==============================] - 4s 28ms/step - loss: 0.3021 - acc: 0.9126 - val_loss: 0.5214 - val_acc: 0.8630
Epoch 6/20
157/157 [==============================] - 4s 28ms/step - loss: 0.2555 - acc: 0.9359 - val_loss: 0.7787 - val_acc: 0.8476
Epoch 7/20
157/157 [==============================] - 4s 28ms/step - loss: 0.2043 - acc: 0.9521 - val_loss: 0.7406 - val_acc: 0.8584
Epoch 8/20
157/15

Here is a quick guide on how to navigate your `Tensorboard` dash:

- The Scalars tab shows changes in the loss and metrics over the epochs. It can be used to track other scalar values such as learning rate and training speed.
- The images tab shows the weights. Adjusting the slider displays the weights at various epochs.
- The graph tab shows your model’s layers. You can use this to check if the architecture of the model looks as intended.
- The distribution tab shows the distribution of tensors (_the distribution of the weights and biases over each epoch_).
- The histograms tab show the distribution of tensors over time.

If you installed TensorBoard via pip, you can launch it via the command line:

> `tensorboard --logdir=logs`

On a Notebook, you can launch it using the cell below.


In [ ]:
from tensorboard import notebook

notebook.display(port=6006, height=1000)

---

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).
